#### Single simulation values

In [19]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Creating interactive widgets for dynamic input with adjusted layout and style
style = {'description_width': 'initial'}  # Adjust the label width to fit the full description


# Creating interactive widgets for dynamic input
initial_inv = widgets.FloatText(value=0, description='Initial Investment:', style=style)
monthly_con = widgets.FloatText(value=40000, description='Monthly Contribution:', style=style)
n_inv = widgets.IntText(value=20, description='Years of Investment:', style=style)
percent_hike = widgets.FloatText(value=0.00, description='Percent Hike (%):', style=style)

rate = widgets.FloatText(value=12, description='Annual Growth Rate (%):', style=style)
rate_std = widgets.FloatText(value=4, description='Growth Rate Std Dev (%):', style=style)

inf_rate = widgets.FloatText(value=6.5, description='Inflation Rate (%):', style=style)
inf_rate_std = widgets.FloatText(value=1.5, description='Inflation Rate Std Dev (%):', style=style)

rate_cons = widgets.FloatText(value=8, description='Conservative Growth Rate (%):', style=style)
year_amount_withdrawn = widgets.FloatText(value=600000, description='Yearly Withdrawal:', style=style)
life_inf = widgets.FloatText(value=1, description='Life style inflation (%):', style=style)

# Output widget for displaying results
output = widgets.Output()

# Displaying widgets
display(initial_inv, monthly_con, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std, rate_cons, year_amount_withdrawn,life_inf)

def calculate_investments(initial_inv, monthly_con, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std, rate_cons, year_amount_withdrawn,life_inf):
    with output:
        clear_output(wait=True)
        
        # Generate random rates and inflation
        rate_list = np.random.normal(rate, rate_std, n_inv)
        rate_list_pm = (pow(1 + rate_list / 100, 1 / 12) - 1) * 100  # Changing percentage rate into per month
        inf_rate_list = np.random.normal(inf_rate, inf_rate_std, n_inv)

        for x in range(n_inv):
            year_amount_withdrawn = year_amount_withdrawn * (1 + inf_rate_list[x] / 100)
        print('Your monthly expense at the end of', n_inv, 'years will be', year_amount_withdrawn / 12)

        print("Average inflation rate for this simulation is", inf_rate_list.mean())
        print("Average growth rate for this simulation is", rate_list.mean())

        total = initial_inv
        for i in range(n_inv):
            monthly_con = monthly_con * (1 + percent_hike/100)

            for j in range(12):
                total = (total + monthly_con) * (1 + rate_list_pm[i] / 100)

        total_investment = monthly_con * 12 * n_inv + initial_inv
        print('Total amount invested -', total_investment)
        print('Total amount in investments -', total)
        print('Latest monthly contribution -', monthly_con)

        years_covered = 0
        while total > 0:
            total = total - year_amount_withdrawn
            total = total * (1 + rate_cons / 100)
            year_amount_withdrawn = year_amount_withdrawn * (1 + np.random.normal(inf_rate, inf_rate_std) / 100)*(1+life_inf/100)
            years_covered += 1

        print("Total years before exhaustion", years_covered)

# Button to execute the calculation
calc_button = widgets.Button(description="Calculate")

def on_button_click(b):
    calculate_investments(
        initial_inv.value, monthly_con.value, n_inv.value, percent_hike.value,
        rate.value, rate_std.value, inf_rate.value, inf_rate_std.value,
        rate_cons.value, year_amount_withdrawn.value,life_inf.value
    )

calc_button.on_click(on_button_click)

# Display button and output widget
display(calc_button, output)


FloatText(value=0.0, description='Initial Investment:', style=DescriptionStyle(description_width='initial'))

FloatText(value=40000.0, description='Monthly Contribution:', style=DescriptionStyle(description_width='initia…

IntText(value=20, description='Years of Investment:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Percent Hike (%):', style=DescriptionStyle(description_width='initial'))

FloatText(value=12.0, description='Annual Growth Rate (%):', style=DescriptionStyle(description_width='initial…

...

FloatText(value=8.0, description='Conservative Growth Rate (%):', style=DescriptionStyle(description_width='in…

FloatText(value=600000.0, description='Yearly Withdrawal:', style=DescriptionStyle(description_width='initial'…

FloatText(value=1.0, description='Life style inflation (%):', style=DescriptionStyle(description_width='initia…

Button(description='Calculate', style=ButtonStyle())

Output()

#### N simulation values

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
import math
from mpmath import mp
from collections import deque
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display, clear_output

# Creating interactive widgets for dynamic input with adjusted layout and style
style = {'description_width': 'initial'}  # Adjust the label width to fit the full description


# Creating interactive widgets for dynamic input
N = widgets.IntText(value=10000, description='Number of simulations:', style=style)
initial_inv = widgets.FloatText(value=0, description='Initial Investment:', style=style)
monthly_sip = widgets.FloatText(value=40000, description='Monthly Contribution:', style=style)
n_inv = widgets.IntText(value=20, description='Years of Investment:', style=style)
percent_hike = widgets.FloatText(value=4, description='Percent Hike (%):', style=style)

rate = widgets.FloatText(value=12, description='Annual Growth Rate (%):', style=style)
rate_std = widgets.FloatText(value=4, description='Growth Rate Std Dev (%):', style=style)

inf_rate = widgets.FloatText(value=6.5, description='Inflation Rate (%):', style=style)
inf_rate_std = widgets.FloatText(value=1.5, description='Inflation Rate Std Dev (%):', style=style)

rate_cons = widgets.FloatText(value=8, description='Conservative Growth Rate (%):', style=style)
current_annual_exp = widgets.FloatText(value=600000, description='Current Annual Expenses:', style=style)
life_inf = widgets.FloatText(value=1, description='Life style inflation (%):', style=style)

# Output widget for displaying results
output = widgets.Output()

# Displaying widgets
display(N,initial_inv, monthly_sip, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std, rate_cons, current_annual_exp,life_inf)

def calculate_investments(N,initial_inv, monthly_sip, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std, rate_cons, current_annual_exp,life_inf):
    with output:
        clear_output(wait=True)
        
        simulation_list = deque()
        
        for _ in range(N):

            year_amount_withdrawn = current_annual_exp
            monthly_con = monthly_sip

            rate_list = np.random.normal(rate,rate_std,n_inv)
            rate_list_pm = (pow(1+rate_list/100, 1/12)-1)*100 #Changing percentage rate into per month

            inf_rate_list = np.random.normal(inf_rate,inf_rate_std,n_inv)


            for x in range(n_inv):
                year_amount_withdrawn = year_amount_withdrawn*(1+inf_rate_list[x]/100)

            # Initialize total with a larger value
            total =  initial_inv
            for i in range(n_inv):
                monthly_con = monthly_con*(1+percent_hike/100)

                for j in range(12):
                    total = (total+monthly_con)*(1+rate_list_pm[i]/100)    

        #     total_investment = monthly_con*12*n_inv+initial_inv

            years_covered = 0
            while total > 0:
                skip = 0
                if total > 1e308 or year_amount_withdrawn > 1e308:  # Example limit, adjust as needed
                    skip = 1
                    break
                total = total - year_amount_withdrawn
                total = total * (1 +  rate_cons/100)
                year_amount_withdrawn = year_amount_withdrawn * (1 + np.random.normal(inf_rate, inf_rate_std)/100)*(1+life_inf/100)
                years_covered += 1

            if skip==0:    
                simulation_list.append(years_covered)

        simulation_list = np.array(simulation_list)

        # # Generate a 1D numpy array (replace this with your own data)
        data = simulation_list

        # Plot the histogram
        plt.hist(data, bins=30, density=True, alpha=0.6, color='g')

        # Fit a normal distribution to the data
        mu, std = norm.fit(data)
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std)
        plt.plot(x, p, 'k', linewidth=2)

        # Add labels and a title
        plt.title("Fit results: mu = %.2f,  std = %.2f" % (mu, std))
        plt.xlabel('Value')
        plt.ylabel('Frequency')

        # Show the plot
        plt.show()

        
# Button to execute the calculation
calc_button = widgets.Button(description="Calculate")

def on_button_click(b):
    calculate_investments(
        N.value,initial_inv.value, monthly_sip.value, n_inv.value, percent_hike.value,
        rate.value, rate_std.value, inf_rate.value, inf_rate_std.value,
        rate_cons.value, current_annual_exp.value,life_inf.value
    )

calc_button.on_click(on_button_click)

# Display button and output widget
display(calc_button, output)
        

IntText(value=10000, description='Number of simulations:', style=DescriptionStyle(description_width='initial')…

FloatText(value=0.0, description='Initial Investment:', style=DescriptionStyle(description_width='initial'))

FloatText(value=40000.0, description='Monthly Contribution:', style=DescriptionStyle(description_width='initia…

IntText(value=20, description='Years of Investment:', style=DescriptionStyle(description_width='initial'))

FloatText(value=4.0, description='Percent Hike (%):', style=DescriptionStyle(description_width='initial'))

...

FloatText(value=8.0, description='Conservative Growth Rate (%):', style=DescriptionStyle(description_width='in…

FloatText(value=600000.0, description='Current Annual Expenses:', style=DescriptionStyle(description_width='in…

FloatText(value=1.0, description='Life style inflation (%):', style=DescriptionStyle(description_width='initia…

Button(description='Calculate', style=ButtonStyle())

Output()

#### Simulate SWP Corpus movement LIVE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
import math
import dash
from dash.dependencies import Output, Input, State
from dash import dcc
from dash import html
import plotly
import random
import plotly.graph_objs as go
# Global variables for storing the state
X = []
Y = []
total = 0
years = 0
simulation_running = False

monthly_contribution = 0
yearly_withdrawal = 0

app = dash.Dash(__name__)

app.layout = html.Div(
    [
        html.H1("Investment Growth Simulation"),
        html.Div([
            html.Label("Initial Investment:"),
            dcc.Input(id='initial_inv', type='number', value=0, step=1000),
        ]),
        html.Div([
            html.Label("Monthly Contribution:"),
            dcc.Input(id='monthly_con', type='number', value=50000, step=1000),
        ]),
        html.Div([
            html.Label("Invest monthly for (years):"),
            dcc.Input(id='n_inv', type='number', value=20, step=1),
        ]),
        html.Div([
            html.Label("Annual Contribution Hike (%):"),
            dcc.Input(id='percent_hike', type='number', value=4.0, step=0.1),
        ]),
        html.Div([
            html.Label("Annual Rate of Return (%):"),
            dcc.Input(id='rate', type='number', value=12, step=0.1),
        ]),
        html.Div([
            html.Label("Rate of return-Standard Deviation (%):"),
            dcc.Input(id='rate_std', type='number', value=4, step=0.1),
        ]),
        html.Div([
            html.Label("Inflation Rate (%):"),
            dcc.Input(id='inf_rate', type='number', value=6.5, step=0.1),
        ]),
        html.Div([
            html.Label("Inflation Rate Standard Deviation (%):"),
            dcc.Input(id='inf_rate_std', type='number', value=2.5, step=0.1),
        ]),
        html.Div([
            html.Label("Conservative Growth Rate (after investment period) (%):"),
            dcc.Input(id='rate_cons', type='number', value=8, step=0.1),
        ]),
        html.Div([
            html.Label("Initial Annual Withdrawal Amount:"),
            dcc.Input(id='year_amount_withdrawn', type='number', value=600000, step=10000),
        ]),
        html.Button('Start Simulation', id='start-button', n_clicks=0),
        html.Button('Reset Simulation', id='reset-button', n_clicks=0),
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*500,
            n_intervals=0
        ),
    ]
)

@app.callback(
    [Output('live-graph', 'figure'),
     Output('start-button', 'n_clicks'),
     Output('reset-button', 'n_clicks')],
    [Input('graph-update', 'n_intervals'),
     Input('start-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')],
    [State('initial_inv', 'value'),
     State('monthly_con', 'value'),
     State('n_inv', 'value'),
     State('percent_hike', 'value'),
     State('rate', 'value'),
     State('rate_std', 'value'),
     State('inf_rate', 'value'),
     State('inf_rate_std', 'value'),
     State('rate_cons', 'value'),
     State('year_amount_withdrawn', 'value')]
)
def update_graph_scatter(n_intervals, start_clicks, reset_clicks,
                         initial_inv, monthly_con, n_inv, percent_hike,
                         rate, rate_std, inf_rate, inf_rate_std,
                         rate_cons, year_amount_withdrawn):
    global X, Y, total, years, simulation_running, monthly_contribution, yearly_withdrawal
    
    if monthly_contribution == 0:
        monthly_contribution = monthly_con
    if yearly_withdrawal == 0:
        yearly_withdrawal = year_amount_withdrawn
    total_investment = initial_inv
    
    

    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = None
        
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'reset-button':
        X.clear()
        Y.clear()
        total = 0
        years = 0
        yearly_withdrawal = 0
        monthly_contribution = 0
        simulation_running = False
        return {'data': [], 'layout': go.Layout()}, 0, 0

    if button_id == 'start-button' and not simulation_running:

        total = initial_inv
        years = 0
        
        rate_list = np.random.normal(rate,rate_std,n_inv)
        rate_list_pm = (pow(1+rate_list/100, 1/12)-1)*100 #Changing percentage rate into per month
        inf_rate_list = np.random.normal(inf_rate, inf_rate_std, n_inv)
        
            
        for i in range(n_inv): 
            monthly_contribution = monthly_contribution*(1+percent_hike/100)
            total_investment = total_investment + 12*monthly_contribution

            for j in range(12): #Corpus growing
                total = (total+monthly_contribution)*(1+rate_list_pm[i]/100)   

        
        for x in range(n_inv): #Finding the yearly withdrawal amaount at the end of investment period due to inflation
            yearly_withdrawal = yearly_withdrawal*(1+inf_rate_list[x]/100)
        
        X.append(years)
        Y.append(total)

        simulation_running = True

    if simulation_running:
        while total > 0 :
            
            X.append(X[-1] + 1)
            total = total - yearly_withdrawal
            total = total*(1 + rate_cons/100)
            Y.append(total)
            yearly_withdrawal = yearly_withdrawal*(1+np.random.normal(inf_rate,inf_rate_std)/100) #yearly withdrawel keeps increasing with time
            print(yearly_withdrawal)
            print(total)


            data = go.Scatter(
                x=list(X),
                y=list(Y),
                name='Scatter',
                mode='lines+markers'
            )

            return {'data': [data], 'layout': go.Layout(
                xaxis=dict(rangeselector=dict(buttons=list([dict(step="all")])),
                           rangeslider=dict(visible=False)),
                yaxis=dict(range=[0.83*min(Y), 1.2*max(Y)])
            )}, 0, 0
    
    else:
        return {'data': [], 'layout': go.Layout()}, 0, 0

if __name__ == '__main__':
    app.run_server(debug=False)


#### Comparing differences between monthly and yearly SIP and Lump sum investing

In [18]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Creating interactive widgets for dynamic input with adjusted layout and style
style = {'description_width': 'initial'}  # Adjust the label width to fit the full description


# Creating interactive widgets for dynamic input
monthly_con = widgets.FloatText(value=40000, description='Monthly Contribution:', style=style)
yearly_con = widgets.FloatText(value=480000, description='Yearly Contribution:', style=style)
lump = widgets.FloatText(value=2500000, description='One time lump sum investment:', style=style)
n_inv = widgets.IntText(value=20, description='Years of Investment:', style=style)
percent_hike = widgets.FloatText(value=1.5, description='Percent Hike (%):', style=style)

rate = widgets.FloatText(value=12, description='Annual Growth Rate (%):', style=style)
rate_std = widgets.FloatText(value=4, description='Growth Rate Std Dev (%):', style=style)

inf_rate = widgets.FloatText(value=6.5, description='Inflation Rate (%):', style=style)
inf_rate_std = widgets.FloatText(value=1.5, description='Inflation Rate Std Dev (%):', style=style)


# Output widget for displaying results
output = widgets.Output()

# Displaying widgets
display( monthly_con, yearly_con, lump, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std)

def calculate_investments( monthly_con, yearly_con, lump, n_inv, percent_hike, rate, rate_std, inf_rate, inf_rate_std):
    with output:
        clear_output(wait=True)
        
        monthly_SIP_corpus = []
        yearly_SIP_corpus = []
        lump_corpus = []
        
        # Generate random rates and inflation
        rate_list = np.random.normal(rate, rate_std, n_inv)
        rate_list_pm = (pow(1 + rate_list / 100, 1 / 12) - 1) * 100  # Changing percentage rate into per month
        inf_rate_list = np.random.normal(inf_rate, inf_rate_std, n_inv)

        print("Average inflation rate for this simulation is", inf_rate_list.mean())
        print("Average growth rate for this simulation is", rate_list.mean())
        print("\n")

        total_M_SIP = 0
        total_investment_M = 0
        for i in range(n_inv):
            monthly_con = monthly_con * (1 + percent_hike/100)

            for j in range(12):
                total_M_SIP = (total_M_SIP + monthly_con) * (1 + rate_list_pm[i] / 100)

            total_investment_M = total_investment_M + monthly_con * 12
            monthly_SIP_corpus.append(total_M_SIP)
            
        print('Total amount invested through monthly SIP-', total_investment_M)
        print('Total amount in investments through monthly SIP -', total_M_SIP)
        print('Latest monthly contribution -', monthly_con)
        print("\n")
        
        
        total_Y_SIP = 0
        total_investment_Y =0
        total_lump = 0
        for i in range(n_inv):
            
            yearly_con = yearly_con * (1 + percent_hike/100)
            
            total_Y_SIP = (total_Y_SIP + yearly_con) * (1 + rate_list[i] / 100)
            lump = lump*(1 + rate_list[i] / 100)
            
            total_investment_Y = total_investment_Y + yearly_con
            yearly_SIP_corpus.append(total_Y_SIP)
            lump_corpus.append(lump)
            
        print('Total amount invested through yearly SIP-', total_investment_Y)
        print('Total amount in investments through yearly SIP -', total_Y_SIP)
        print('Latest yearly contribution -', yearly_con)
        print('\nTotal amount in investments through lump growth -', lump)
        
        plt.plot(range(1,n_inv+1), monthly_SIP_corpus, label='Monthly')
        plt.plot(range(1,n_inv+1), yearly_SIP_corpus, label='Yearly')
        plt.plot(range(1,n_inv+1), lump_corpus, label='Lump')


        # Adding labels and legend
        plt.xlabel('X-axis')
        plt.ylabel('Y-axis')
        plt.legend()

        # Show the plot
        plt.show()

# Button to execute the calculation
calc_button = widgets.Button(description="Calculate")

def on_button_click(b):
    calculate_investments(
        monthly_con.value, yearly_con.value, lump.value, n_inv.value, percent_hike.value,
        rate.value, rate_std.value, inf_rate.value, inf_rate_std.value
    )

calc_button.on_click(on_button_click)

# Display button and output widget
display(calc_button, output)


FloatText(value=40000.0, description='Monthly Contribution:', style=DescriptionStyle(description_width='initia…

FloatText(value=480000.0, description='Yearly Contribution:', style=DescriptionStyle(description_width='initia…

FloatText(value=2500000.0, description='One time lump sum investment:', style=DescriptionStyle(description_wid…

IntText(value=20, description='Years of Investment:', style=DescriptionStyle(description_width='initial'))

FloatText(value=1.5, description='Percent Hike (%):', style=DescriptionStyle(description_width='initial'))

...

FloatText(value=4.0, description='Growth Rate Std Dev (%):', style=DescriptionStyle(description_width='initial…

FloatText(value=6.5, description='Inflation Rate (%):', style=DescriptionStyle(description_width='initial'))

FloatText(value=1.5, description='Inflation Rate Std Dev (%):', style=DescriptionStyle(description_width='init…

Button(description='Calculate', style=ButtonStyle())

Output()